# What Happened to the Quants in August 2007?
### Implementation of Khandani & Lo (2007) with Extension to February 2026

> **Paper**: Khandani, A.E. and Lo, A.W. (2007). *"What Happened to the Quants in August 2007?"*
> First Draft: September 20, 2007. MIT Laboratory for Financial Engineering.

---
**Notebook covers:**
- Comprehensive paper notes and implementation plan
- Core contrarian strategy (Equations 1–3)
- Leverage choice (Equations 4–5) — Table 6 reproduction
- Table 7: Leveraged daily returns for August 2007 (8:1 leverage)
- Figure 4: Leveraged contrarian strategy vs market indexes (August 2007)
- Extension to February 2026 with downloadable live data

## 📋 Comprehensive Paper Notes

### 1. Background & Context
The week of August 6, 2007 witnessed unprecedented losses for quantitative long/short equity hedge funds. Unlike the Bear Stearns and Sowood losses (credit-related), these losses hit **market-neutral** strategies that were supposed to be immune to market moves. The losses were confined almost exclusively to **quantitatively-driven** funds, hitting on Aug 7–8 when the S&P 500 actually *rose* both days.

### 2. Key Hypotheses (The "Unwind Hypothesis")
The authors argue the losses were caused by a rapid cascade:
1. **Initial trigger**: A large quant equity market-neutral portfolio was forcibly liquidated (likely margin calls from deteriorating credit/subprime positions).
2. **Price impact**: The rapid selling of "winner" stocks and buying of "loser" stocks caused temporary but severe price dislocations.
3. **Cascade**: Other similarly-constructed portfolios experienced losses → triggered stop-losses → more de-leveraging → more price impact.
4. **Reversal on Aug 10**: A large (+23.67% leveraged) reversal confirmed it was a liquidity trade, not a fundamental shift.

### 3. The Contrarian Strategy
First proposed by Lehmann (1990) and Lo & MacKinlay (1990). It is a **short-term mean-reversion** strategy:
- **Universe**: All U.S. common stocks (CRSP codes 10 & 11), price $5–$2,000
- **Signal**: Yesterday's losers are bought; yesterday's winners are sold
- **Rebalancing**: Daily
- **Structure**: Dollar-neutral (equal dollar long and short), market-neutral

Key insight: This strategy **provides liquidity** to the market and profits from mean reversion and positive cross-autocorrelations across stocks.

### 4. Alpha Decay (The Core Problem)
| Year | Avg Daily Return | Commentary |
|------|-----------------|------------|
| 1995 | 1.38% | Enormous — frictionless theoretical max |
| 1998 | 0.57% | Paper's baseline year |
| 2003 | 0.21% | Post-dot-com decline |
| 2006 | 0.15% | Near-floor profitability |
| 2007 | 0.13% | Almost 10× decline from 1995 |

**Why the decline?** Increased competition, decimalization (2001), lower volatility, better technology, more capital chasing same signals.

### 5. The Leverage Trap (Section 6)
As alpha decayed, managers **increased leverage** to maintain target returns. Using 1998 as baseline:
- **Equation 4**: Set E[L_pt] = (θ/2)·E[R_pt] = E[R_p,1998]
- **Equation 5**: θ* = 2·E[R_p,1998] / E[R_pt]

By 2006, the required leverage was ~**8:1** (vs 2:1 Reg-T). By 2007 (ex-ante), ~**9:1**.
This is why a 12-σ unleveraged event became a ~48-σ leveraged disaster.

### 6. August 2007 vs August 1998
A crucial comparison: in August 1998 (LTCM crisis, Russia default), the contrarian strategy showed **no unusual behavior** — it was profitable and calm every day. In August 2007, it lost 12 daily standard deviations in 3 days.

This signals dramatically increased **financial market integration** and systemic risk by 2007. The LTCM crisis was contained in fixed-income; the 2007 crisis crossed all asset classes.

### 7. Crowded Trade (Figure 3 / TASS Data)
- Long/Short Equity Hedge + Equity Market Neutral AUM grew from ~$20B (1995) to >$160B (2007)
- Average assets per fund grew from $62MM (1994) to $229MM (2007) — exponentially
- More capital → more competition → lower alpha → higher required leverage → higher systemic risk

### 8. The Leveraged Returns (Table 7 / Figure 4)
Using θ = 8 (the 2006 estimated leverage level), multiplied by θ/2 = 4:
| Date | Unleveraged | Leveraged (8:1) | S&P 500 |
|------|-------------|-----------------|---------|
| Aug 7 | -1.16% | **-4.64%** | +0.62% |
| Aug 8 | -2.83% | **-11.33%** | +1.44% |
| Aug 9 | -2.86% | **-11.43%** | -2.95% |
| Aug 10 | +5.92% | **+23.67%** | +0.04% |

Funds that capitulated on Aug 8–9 missed the reversal on Aug 10.

### 9. Systemic Risk Implications
- Greater hedge-fund connectivity (higher pairwise correlations across CS/Tremont indexes)
- Increasing illiquidity of supposedly liquid long/short equity funds (positive autocorrelation in monthly returns)
- 130/30 strategies adding $75B+ of long/short activity with similar factor exposures
- Authors conclude the "Doomsday Clock" for systemic risk stands at ~11:51pm

### 10. Implementation Notes for This Notebook
- **1995–2007**: Uses exact data from the paper (Tables 2, 3, 6, 7)
- **2008–2026**: Uses yfinance data (S&P 500 proxy universe — ~500 stocks vs ~4,000 in CRSP)
- **Key caveat**: S&P 500 is survivorship-biased and larger-cap than CRSP; small-cap mean reversion is understated
- **Leverage baseline**: Maintained at 1998 level (E[Rp,1998] = 0.57%/day) throughout

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore')

# ── Plotting style ──────────────────────────────────────────
plt.rcParams.update({
    'figure.dpi': 120,
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
    'lines.linewidth': 1.8,
    'grid.alpha': 0.3,
    'axes.spines.top': False,
    'axes.spines.right': False,
})

print("✅ Imports complete.")
print("Python version:", __import__('sys').version.split()[0])
print("pandas:", pd.__version__, "| numpy:", np.__version__)

## Part I: Core Strategy Functions (Equations 1–5)

In [ ]:
# ══════════════════════════════════════════════════════════
#  EQUATION 1 — Contrarian Portfolio Weights
# ══════════════════════════════════════════════════════════
# ω_{it} = -(1/N) · (R_{i,t-1} − R_{m,t-1})
# R_{m,t-1} = (1/N) · Σ R_{i,t-1}   [equal-weighted market return]

def contrarian_weights(prev_returns: pd.Series) -> pd.Series:
    """
    Compute Equation (1): contrarian portfolio weights.
    prev_returns : Series of stock returns at t-1 (already cleaned/filtered).
    Returns      : Series of weights (dollar-neutral by construction).
    """
    prev_returns = prev_returns.dropna()
    N = len(prev_returns)
    if N == 0:
        return pd.Series(dtype=float)
    Rm = prev_returns.mean()          # equal-weighted market return
    omega = -(1.0 / N) * (prev_returns - Rm)
    return omega


# ══════════════════════════════════════════════════════════
#  EQUATION 2 — Gross Investment & Unleveraged Return
# ══════════════════════════════════════════════════════════
# I_t = (1/2) · Σ |ω_{it}|
# R_pt = Σ ω_{it}·R_{it} / I_t

def unleveraged_return(omega: pd.Series, curr_returns: pd.Series) -> float:
    """
    Compute Equation (2): unleveraged portfolio return for one day.
    omega        : weights from contrarian_weights()
    curr_returns : current-day returns for the same stocks
    Returns      : scalar unleveraged return R_pt
    """
    # Align on common tickers, drop any NaN
    common = omega.index.intersection(curr_returns.dropna().index)
    omega_c   = omega.loc[common]
    returns_c = curr_returns.loc[common]

    It = 0.5 * omega_c.abs().sum()    # gross investment
    if It == 0:
        return np.nan
    profit = (omega_c * returns_c).sum()
    return profit / It


# ══════════════════════════════════════════════════════════
#  EQUATION 3 — Leveraged Return
# ══════════════════════════════════════════════════════════
# L_pt(θ) = (θ/2) · R_pt

def leveraged_return(Rpt: float, theta: float) -> float:
    """Compute Equation (3): leveraged return given leverage ratio θ:1."""
    return (theta / 2.0) * Rpt


# ══════════════════════════════════════════════════════════
#  EQUATIONS 4 & 5 — Optimal Leverage Choice
# ══════════════════════════════════════════════════════════
# Target: E[L_pt] = (θ*/2)·E[R_pt] = E[R_p,1998]
# Solution: θ* = 2·E[R_p,1998] / E[R_pt]

E_Rp_1998 = 0.57 / 100.0   # 0.57% per day → decimal

def optimal_leverage(E_Rpt: float,
                     E_Rp_baseline: float = E_Rp_1998) -> tuple:
    """
    Compute Equations (4)+(5): optimal leverage and return multiplier.
    E_Rpt           : average daily return in year t (decimal)
    E_Rp_baseline   : baseline expected daily return (1998 level)
    Returns: (return_multiplier, leverage_ratio θ*)
    """
    if E_Rpt <= 0:
        return np.nan, np.nan
    multiplier = E_Rp_baseline / E_Rpt          # return multiplier
    theta_star = 2.0 * E_Rp_baseline / E_Rpt   # Eq. (5)
    return multiplier, theta_star


# ══════════════════════════════════════════════════════════
#  VECTORISED STRATEGY — Apply to a price DataFrame
# ══════════════════════════════════════════════════════════
def compute_contrarian_strategy(
        prices_df: pd.DataFrame,
        min_price: float = 5.0,
        max_price: float = 2000.0) -> pd.Series:
    """
    Apply contrarian strategy to a price DataFrame in a vectorised way.

    prices_df : DataFrame, index=dates, columns=ticker symbols, values=close prices
    Returns   : Series of UNLEVERAGED daily returns (R_pt)

    Algorithm:
      1. Compute daily pct returns.
      2. For each day t, use t-1 prices to create valid-stock mask ($5-$2,000).
      3. Compute equal-weighted market return of valid stocks at t-1.
      4. Weights ω_{it} = -(R_{i,t-1} − Rm_{t-1}) / N   [Eq.1]
      5. I_t = 0.5·Σ|ω_{it}|                              [Eq.2 numerator]
      6. R_pt = Σ(ω_{it}·R_{it}) / I_t                    [Eq.2 return]
    """
    prices = prices_df.copy()

    # Step 1: returns
    rets = prices.pct_change()

    # Step 2: price validity mask (based on PREVIOUS day's closing prices)
    prev_prices = prices.shift(1)
    valid = (prev_prices >= min_price) & (prev_prices <= max_price)

    # Mask returns to valid stocks only
    r_prev = rets.shift(1).where(valid)    # R_{i,t-1} (previous day, valid stocks)
    r_curr = rets.where(valid)             # R_{i,t}   (current day, same mask)

    # Step 3: equal-weighted market return at t-1
    N      = valid.sum(axis=1)             # number of valid stocks per day
    Rm     = r_prev.mean(axis=1)          # mean of r_prev (skipna by default)

    # Step 4: weights  ω_{it} = -(R_{i,t-1} − Rm_{t-1}) / N
    excess  = r_prev.subtract(Rm, axis=0) # shape (T, N)
    omega   = -excess.div(N, axis=0)

    # Step 5: gross investment
    It = 0.5 * omega.abs().sum(axis=1)

    # Step 6: profit and return
    profit = (omega * r_curr).sum(axis=1)
    Rpt    = profit / It
    Rpt    = Rpt.replace([np.inf, -np.inf], np.nan)

    return Rpt


print("✅ Core functions defined (Equations 1-5).")
print(f"   Baseline 1998 average daily return: {E_Rp_1998:.4%}")

## Part II: Reproducing Paper Results (1995–2007)

### Table 2 — Annual Statistics of Contrarian Strategy (All Stocks)

In [ ]:
# ── Table 2: Year-by-Year Average Daily Returns (from paper, "All" column) ──
paper_years = list(range(1995, 2008))

# Average daily return (%) — "All" column, Table 2
paper_avg_daily_ret = {
    1995: 1.38, 1996: 1.17, 1997: 0.88, 1998: 0.57, 1999: 0.44,
    2000: 0.44, 2001: 0.31, 2002: 0.45, 2003: 0.21, 2004: 0.37,
    2005: 0.26, 2006: 0.15, 2007: 0.13,
}
# Standard deviation of daily returns (%) — "All" column, Table 2
paper_std_daily_ret = {
    1995: 0.40, 1996: 0.48, 1997: 0.68, 1998: 0.84, 1999: 1.02,
    2000: 1.68, 2001: 1.43, 2002: 0.98, 2003: 0.54, 2004: 0.53,
    2005: 0.46, 2006: 0.52, 2007: 0.72,
}
# Annualised Sharpe (√250 × mean/std) — "All" column, Table 2
paper_sharpe = {
    1995: 53.87, 1996: 38.26, 1997: 20.46, 1998: 10.62, 1999: 6.81,
    2000:  4.17, 2001:  3.46, 2002:  7.25, 2003:  5.96, 2004: 11.07,
    2005:  8.85, 2006:  4.47, 2007:  2.79,
}

df_table2 = pd.DataFrame({
    'Year':           paper_years,
    'Avg_Daily_Ret%': [paper_avg_daily_ret[y] for y in paper_years],
    'Std_Daily_Ret%': [paper_std_daily_ret[y] for y in paper_years],
    'Ann_Sharpe':     [paper_sharpe[y]         for y in paper_years],
}).set_index('Year')

print("Table 2 — Annual Statistics (Paper Data, 'All Stocks' column)")
print("="*62)
print(df_table2.to_string())

# Quick verification of Sharpe formula
df_table2['Sharpe_Computed'] = (
    df_table2['Avg_Daily_Ret%'] / df_table2['Std_Daily_Ret%'] * np.sqrt(250)
)
print("\nSharpe ratio cross-check (computed vs paper) — max discrepancy:",
      (df_table2['Sharpe_Computed'] - df_table2['Ann_Sharpe']).abs().max().round(2))

In [ ]:
# ── Figure 1: Secular Decline in Average Daily Returns ──────────────────────
fig, ax = plt.subplots(figsize=(11, 5))
ax.bar(df_table2.index, df_table2['Avg_Daily_Ret%'],
       color='steelblue', edgecolor='white', alpha=0.85, label='Avg Daily Return (All)')
ax.axhline(0.57, color='red', linestyle='--', linewidth=1.2, label='1998 baseline (0.57%/day)')
ax.set_xlabel("Year")
ax.set_ylabel("Average Daily Return (%)")
ax.set_title("Figure 1: Secular Decline of Contrarian Strategy Returns, 1995-2007\n(Reproducing Khandani & Lo 2007, Figure 1 concept)")
ax.legend()
ax.set_xticks(paper_years)
ax.tick_params(axis='x', rotation=45)
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f%%'))
plt.tight_layout()
plt.savefig('fig1_alpha_decay.png', dpi=150, bbox_inches='tight')
plt.show()
print("✅ Alpha decay clearly visible: 1.38%/day (1995) → 0.13%/day (2007)")

### Table 6 — Leverage Required to Match 1998 Returns (Equations 4 & 5)

**Equation 4** (target return condition):
$$E[L_{pt}] \;=\; \frac{\theta^*}{2} \cdot E[R_{pt}] \;=\; E[R_{p,1998}]$$

**Equation 5** (optimal leverage):
$$\theta^* \;=\; \frac{2 \cdot E[R_{p,1998}]}{E[R_{pt}]}, \quad t = 1999, 2000, \ldots$$

We use 1998 as baseline because it represents the "expectations level" that investors had before alpha decay became severe. The factor of 2 arises from the definition of leverage (sum of absolute long + short positions ÷ capital).

In [ ]:
# ── Table 6: Leverage Calculation (Equations 4 & 5) ───────────────────────
E_Rp_1998_pct = 0.57   # percent

# Paper Table 6 (exact) — includes these years
table6_years = [1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007]
table6_returns = {
    1998: 0.57, 1999: 0.44, 2000: 0.44, 2001: 0.31,
    2002: 0.45, 2003: 0.21, 2004: 0.37, 2005: 0.26,
    2006: 0.15, 2007: 0.13,
}

df_table6 = pd.DataFrame({
    'Year':             table6_years,
    'Avg_Daily_Ret%':   [table6_returns[y] for y in table6_years],
}).set_index('Year')

# Equations 4 & 5
df_table6['Return_Multiplier'] = E_Rp_1998_pct / df_table6['Avg_Daily_Ret%']   # 1/scalar
df_table6['Leverage_Ratio_θ*'] = 2.0 * E_Rp_1998_pct / df_table6['Avg_Daily_Ret%']  # Eq.5

print("Table 6 — Leverage Required to Match 1998 Expected Return")
print("(E[R_p,1998] = 0.57%/day as baseline)")
print("="*65)
header = f"{'Year':>6} {'Avg Daily Return':>18} {'Return Multiplier':>18} {'Leverage Ratio θ*':>18}"
print(header)
print("-"*65)
for yr, row in df_table6.iterrows():
    print(f"{yr:>6} {row['Avg_Daily_Ret%']:>17.2f}%   {row['Return_Multiplier']:>17.2f}    {row['Leverage_Ratio_θ*']:>17.2f}:1")

print("\n📌 Paper's exact Table 6 values (cross-check):")
print("   1999: multiplier=1.28, leverage=2.57")
print("   2006: multiplier=3.88, leverage=7.76")
print("   2007: multiplier=4.48, leverage=8.96")
print(f"\n   Our 2006: {df_table6.loc[2006,'Return_Multiplier']:.2f}, {df_table6.loc[2006,'Leverage_Ratio_θ*']:.2f}:1  ✓")
print(f"   Our 2007: {df_table6.loc[2007,'Return_Multiplier']:.2f}, {df_table6.loc[2007,'Leverage_Ratio_θ*']:.2f}:1  ✓")

In [ ]:
# ── Leverage Plot ──────────────────────────────────────────────────────────
fig, ax1 = plt.subplots(figsize=(11, 5))

color1, color2 = 'steelblue', 'tomato'
ax1.bar(df_table6.index, df_table6['Avg_Daily_Ret%'],
        color=color1, alpha=0.7, label='Avg Daily Return % (left)')
ax1.set_xlabel("Year")
ax1.set_ylabel("Average Daily Return (%)", color=color1)
ax1.tick_params(axis='y', labelcolor=color1)
ax1.set_ylim(0, 0.65)

ax2 = ax1.twinx()
ax2.plot(df_table6.index, df_table6['Leverage_Ratio_θ*'],
         color=color2, marker='o', linewidth=2, label='Required Leverage θ* (right)')
ax2.set_ylabel("Required Leverage Ratio θ* (Eq. 5)", color=color2)
ax2.tick_params(axis='y', labelcolor=color2)
ax2.axhline(8, color=color2, linestyle=':', alpha=0.4, linewidth=1)
ax2.text(2006.5, 8.2, 'θ=8 used for Table 7', fontsize=9, color=color2)

ax1.set_title("Equations 4 & 5: As Alpha Decays, Required Leverage Explodes\n(Khandani & Lo 2007, Table 6)")
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')
ax1.set_xticks(table6_years)
plt.tight_layout()
plt.savefig('fig_leverage_buildup.png', dpi=150, bbox_inches='tight')
plt.show()

## Part III: Table 7 & Figure 4 — August 2007 Leveraged Returns

Table 7 = Table 3 × (θ/2) = Table 3 × **4**  (since θ = 8:1 leverage)

We reproduce both the "All Stocks" column and all 10 market-cap decile columns.

In [ ]:
# ── Table 3 Data: Unleveraged Daily Returns — Full Decile Matrix ───────────
# Date, Smallest, D2, D3, D4, D5, D6, D7, D8, D9, Largest, All
table3_raw = [
    ('7/30/2007', -0.07,  0.02,  1.96, -0.36,  0.07,  0.23,  0.26,  0.38,  0.51,  0.18,  0.44),
    ('7/31/2007',  0.19,  1.10,  0.28,  0.55, -0.63,  0.02, -0.80,  0.49, -0.31,  0.06,  0.36),
    ('8/1/2007',   1.53,  0.45, -1.39,  0.35,  0.95, -0.88, -0.71, -0.63, -2.02, -0.22,  0.11),
    ('8/2/2007',   0.88, -0.76, -0.12, -0.67, -0.94, -2.70,  2.16,  1.53, -0.74, -0.19, -0.30),
    ('8/3/2007',  -0.95, -0.62, -0.78,  0.06,  0.88,  0.01, -0.62, -1.09, -0.57, -0.68, -0.02),
    ('8/6/2007',  -0.83, -1.77, -0.39, -1.03,  1.37, -1.37, -1.19, -0.72,  0.27,  0.77,  0.50),
    ('8/7/2007',   0.75,  0.26, -1.64, -2.91, -1.50, -0.70,  0.36, -1.02, -1.72, -0.67, -1.16),
    ('8/8/2007',   0.88, -1.33, -2.59, -3.65, -4.27, -2.16, -2.23, -3.46, -1.26, -1.48, -2.83),
    ('8/9/2007',   0.91, -1.86, -3.87, -2.77, -3.18, -3.95, -3.27, -4.33, -2.58, -1.31, -2.86),
    ('8/10/2007', -0.33,  3.65,  6.08,  7.90,  8.77,  7.67,  7.52,  6.70,  4.68,  2.39,  5.92),
    ('8/13/2007',  1.36, -0.31, -0.63, -1.07, -1.55, -0.22, -1.29, -2.01, -2.14, -1.25, -0.76),
    ('8/14/2007',  1.16,  0.91, -0.26,  0.34,  0.56, -0.28,  0.69, -0.29,  0.16,  0.17,  0.08),
    ('8/15/2007',  0.88,  1.19, -0.61, -0.58, -0.17, -0.97, -0.24, -1.34, -0.57, -1.18, -0.38),
    ('8/16/2007', -1.26, -0.54,  0.15, -0.59, -0.60, -0.99, -1.73, -1.27,  0.27, -1.83, -0.81),
    ('8/17/2007',  3.57,  2.49,  0.10,  1.26,  1.33, -0.52,  0.12, -0.39,  0.31,  0.11,  0.38),
    ('8/20/2007',  3.75,  1.75,  0.35,  1.35,  0.51,  0.44,  1.22,  0.56,  0.39,  1.17,  1.14),
    ('8/21/2007',  1.24,  0.11,  0.01, -0.45,  0.02, -0.63, -0.08, -0.05,  0.19,  0.11,  0.06),
    ('8/22/2007', -0.85, -0.31, -0.52, -0.51, -0.17, -0.83, -0.18, -0.56,  0.39,  0.09, -0.38),
    ('8/23/2007', -0.03,  0.70,  0.70, -0.16,  0.38,  1.04,  0.26, -0.33,  0.32,  0.31,  0.33),
    ('8/24/2007',  0.62, -0.28, -0.07,  0.23,  0.92, -0.06, -0.07,  0.09, -0.35,  0.61,  0.43),
    ('8/27/2007',  1.10,  0.70,  0.11,  0.20,  1.25, -0.16,  0.39,  0.71,  0.71,  0.03,  0.75),
    ('8/28/2007',  0.41,  0.32,  0.08, -0.61, -0.64, -0.50, -0.33, -0.44, -0.47,  0.25, -0.76),
    ('8/29/2007',  1.45,  0.08,  1.27,  2.08,  1.94, -0.53,  1.42,  1.60,  0.91,  0.98,  1.76),
    ('8/30/2007',  1.07,  0.04,  0.62,  0.40,  0.89,  0.10, -0.03, -0.04,  0.12, -0.05,  0.50),
    ('8/31/2007',  1.69,  0.97,  0.95, -0.55,  0.05,  0.52, -0.08, -0.67,  0.01,  0.14,  0.36),
]

cols = ['Date','Smallest','D2','D3','D4','D5','D6','D7','D8','D9','Largest','All']
df_table3 = pd.DataFrame(table3_raw, columns=cols)
df_table3['Date'] = pd.to_datetime(df_table3['Date'])
df_table3 = df_table3.set_index('Date')

# ── Compute Table 7 (Eq. 3 with θ = 8) ─────────────────────────────────────
THETA = 8
MULTIPLIER = THETA / 2   # = 4

df_table7 = df_table3 * MULTIPLIER

print(f"Table 7 = Table 3 × {MULTIPLIER:.0f}  (θ={THETA}:1 leverage, multiplier = θ/2 = {MULTIPLIER:.0f})")
print("="*110)
print(df_table7.round(2).to_string())
print()
# Verify key cells against paper
checks = {
    '8/7/2007 All':  (df_table7.loc['2007-08-07','All'],  -4.64),
    '8/8/2007 All':  (df_table7.loc['2007-08-08','All'], -11.33),
    '8/9/2007 All':  (df_table7.loc['2007-08-09','All'], -11.43),
    '8/10/2007 All': (df_table7.loc['2007-08-10','All'], +23.67),
}
print("\n📌 Cross-checks vs paper (Table 7):")
for label, (computed, expected) in checks.items():
    flag = '✓' if abs(computed - expected) < 0.05 else '✗'
    print(f"   {label}: computed={computed:.2f}%  expected={expected:.2f}%  {flag}")

### Figure 4 — Leveraged Contrarian Strategy vs Market Indexes (August 2007)

Figure 4 in the paper plots the leveraged daily returns of the contrarian strategy (8:1 leverage)
alongside S&P 500, S&P Small Cap 600, Lehman US Gov Aggregate, GSCI, and Trade-Weighted USD.

In [ ]:
# ── Market Index Data for August 2007 (from paper Table 4) ─────────────────
# Date, S&P500, S&P600, MSCI_EM, MSCI_WorldExUS, Lehman_Agg, Lehman_HY, GSCI, USD, VIX_chg
table4_raw = [
    ('7/30/2007',  1.03,  0.94,  0.87,  0.14, -0.04,  0.18,  0.11, -0.12, -3.30),
    ('7/31/2007', -1.26, -0.88,  1.67,  1.36,  0.17,  0.61,  1.18, -0.10,  2.65),
    ('8/1/2007',   0.73,  0.19, -3.42, -1.70,  0.04, -0.15, -1.34,  0.13,  0.15),
    ('8/2/2007',   0.46,  0.98,  0.61,  0.62,  0.04,  0.53,  0.00, -0.20, -2.45),
    ('8/3/2007',  -2.65, -3.48, -0.05, -0.37,  0.29,  0.08, -1.10, -0.66,  3.94),
    ('8/6/2007',   2.42,  1.35, -1.99, -0.57, -0.14, -0.29, -2.76,  0.10, -2.56),
    ('8/7/2007',   0.62,  0.71,  0.45,  0.56, -0.04,  0.38,  0.34,  0.28, -1.04),
    ('8/8/2007',   1.44,  1.52,  2.83,  1.88, -0.48,  0.84, -0.20, -0.17, -0.11),
    ('8/9/2007',  -2.95, -1.38, -1.28, -1.52,  0.31, -0.07, -0.37,  0.54,  5.03),
    ('8/10/2007',  0.04,  1.01, -3.30, -2.85,  0.07, -0.29, -0.03, -0.12,  1.82),
    ('8/13/2007', -0.03, -0.84,  1.01,  1.08,  0.04,  0.34,  0.27,  0.46, -1.73),
    ('8/14/2007', -1.81, -1.87, -1.42, -1.10,  0.23, -0.10,  0.35,  0.54,  1.11),
    ('8/15/2007', -1.36, -1.45, -2.39, -1.52,  0.15, -0.56,  0.80,  0.41,  2.99),
    ('8/16/2007',  0.33,  1.70, -5.63, -2.91,  0.58, -0.59, -3.01, -0.11,  0.16),
    ('8/17/2007',  2.46,  2.30,  0.12,  0.96, -0.28,  0.24,  1.49, -0.37, -0.84),
    ('8/20/2007', -0.03,  0.30,  3.78,  1.23,  0.23,  0.24, -1.65, -0.03, -3.66),
    ('8/21/2007',  0.11,  0.21, -0.18,  0.61,  0.24,  0.19, -1.14,  0.11, -1.08),
    ('8/22/2007',  1.18,  1.19,  2.58,  1.27, -0.16,  0.37,  0.04, -0.30, -2.36),
    ('8/23/2007', -0.11, -1.16,  1.76,  1.16, -0.01,  0.22,  0.96, -0.13, -0.27),
    ('8/24/2007',  1.16,  1.44,  0.44,  0.51, -0.10,  0.04,  1.10, -0.59, -1.90),
    ('8/27/2007', -0.85, -1.07,  1.90,  0.29,  0.23,  0.17,  0.28,  0.09,  2.00),
    ('8/28/2007', -2.34, -2.70, -0.85, -1.26,  0.34, -0.07, -0.17,  0.02,  3.58),
    ('8/29/2007',  2.22,  2.28, -0.23,  0.04, -0.09, -0.06,  1.40, -0.07, -2.49),
    ('8/30/2007', -0.41, -0.38,  1.31,  0.80,  0.29,  0.06,  0.15,  0.12,  1.25),
    ('8/31/2007',  1.12,  1.28,  2.39,  1.58, -0.16,  0.01,  0.48,  0.00, -1.68),
]

idx_cols = ['Date','SP500','SP600','MSCI_EM','MSCI_WorldExUS',
            'Lehman_Agg','Lehman_HY','GSCI','USD','VIX_change']
df_table4 = pd.DataFrame(table4_raw, columns=idx_cols)
df_table4['Date'] = pd.to_datetime(df_table4['Date'])
df_table4 = df_table4.set_index('Date')

# ── Figure 4: Leverage Contrarian Strategy vs Indexes ───────────────────────
# Paper shows: Leveraged Contrarian, S&P 500, S&P 600 Small Cap, Lehman Corp-Gov, GSCI, USD
# Filter to August only (paper's Figure 4 scope)
aug_dates = df_table7.index[df_table7.index >= '2007-08-01']

fig, ax = plt.subplots(figsize=(14, 7))

# Leveraged contrarian (thick, highlighted)
ax.plot(aug_dates, df_table7.loc[aug_dates, 'All'],
        color='navy', linewidth=2.8, marker='o', markersize=5,
        label='Leveraged Contrarian Strategy (8:1)', zorder=5)

# Market indexes
styles = [
    ('SP500',      'S&P 500',          'tomato',   '-',  1.6),
    ('SP600',      'S&P 600 Small Cap','darkorange','-',  1.4),
    ('Lehman_Agg', 'Lehman Corp-Gov',  'green',    '--', 1.4),
    ('GSCI',       'GSCI Commodity',   'purple',   '-.',  1.3),
    ('USD',        'Trade-Wtd USD',    'gray',     ':',   1.3),
]
for col, label, color, ls, lw in styles:
    ax.plot(df_table4.loc[aug_dates].index, df_table4.loc[aug_dates, col],
            color=color, linestyle=ls, linewidth=lw, label=label, alpha=0.85)

# Annotate key events
ax.annotate('Aug 7: −4.64%\n(S&P +0.62%)', xy=(pd.Timestamp('2007-08-07'), -4.64),
            xytext=(pd.Timestamp('2007-08-07'), -8.5),
            arrowprops=dict(arrowstyle='->', color='navy', lw=1.5),
            fontsize=9, color='navy', ha='center')
ax.annotate('Aug 8: −11.33%', xy=(pd.Timestamp('2007-08-08'), -11.33),
            xytext=(pd.Timestamp('2007-08-09'), -14.5),
            arrowprops=dict(arrowstyle='->', color='navy', lw=1.5),
            fontsize=9, color='navy', ha='center')
ax.annotate('Aug 10: +23.67%\n(Reversal)', xy=(pd.Timestamp('2007-08-10'), 23.67),
            xytext=(pd.Timestamp('2007-08-14'), 22),
            arrowprops=dict(arrowstyle='->', color='darkgreen', lw=1.5),
            fontsize=9, color='darkgreen', ha='left')

ax.axhline(0, color='black', linewidth=0.8, alpha=0.5)
ax.axvspan(pd.Timestamp('2007-08-07'), pd.Timestamp('2007-08-09'),
           color='red', alpha=0.06, label='Crisis Days (Aug 7-9)')
ax.set_xlabel("Date (August 2007)")
ax.set_ylabel("Daily Return (%)")
ax.set_title("Figure 4: Leveraged Daily Returns — Contrarian Strategy vs Market Indexes, August 2007\n"
             "(8:1 Leverage, Multiplier=4 — Khandani & Lo 2007, Figure 4 Reproduction)")
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.xaxis.set_major_locator(mdates.WeekdayLocator(byweekday=0))
plt.xticks(rotation=30)
ax.legend(loc='upper right', fontsize=9, framealpha=0.9)
ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.1f%%'))
plt.tight_layout()
plt.savefig('fig4_leveraged_returns_aug2007.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# ── Cumulative Leveraged Returns During August 2007 ─────────────────────────
# Arithmetic compounding (as used in the paper)
crisis_window = df_table7.loc['2007-08-06':'2007-08-10', 'All']
print("=== Cumulative Losses/Gains: Aug 6-10, 2007 (8:1 Leverage) ===")
cumulative = 0.0
for date, ret in crisis_window.items():
    cumulative += ret
    print(f"  {date.strftime('%b %d')}: {ret:+.2f}%  |  Cumulative: {cumulative:+.2f}%")

print()
# Three-day loss Aug 7-9
loss_3d = df_table7.loc['2007-08-07':'2007-08-09', 'All'].sum()
print(f"📉 Three-day cumulative loss (Aug 7-9):  {loss_3d:.2f}%  (≈ −25% of capital)")
print(f"📈 Aug 10 reversal:                     +{df_table7.loc['2007-08-10','All']:.2f}%")
print(f"📊 Full week (Aug 6-10):                {df_table7.loc['2007-08-06':'2007-08-10','All'].sum():.2f}% (arithmetic)")
print()
print("=== Standard Deviation Context ===")
sigma_2006 = 0.52  # percent (paper's 2006 std dev, used as benchmark)
print(f"2006 daily std deviation: {sigma_2006}%")
print(f"Aug 7 unleveraged loss:  {df_table3.loc['2007-08-07','All']:.2f}% → "
      f"{abs(df_table3.loc['2007-08-07','All'])/sigma_2006:.1f}σ")
three_day_unlev = df_table3.loc['2007-08-07':'2007-08-09','All'].sum()
print(f"Aug 7-9 cumulative:      {three_day_unlev:.2f}% → {abs(three_day_unlev)/sigma_2006:.1f}σ  (≈12σ)")

## Part IV: Extension to February 2026

This section downloads live stock data from Yahoo Finance (via `yfinance`) and applies the contrarian strategy to extend the analysis from 2008 to February 2026.

**Key methodological differences from the paper:**
- **Universe**: S&P 500 current constituents (~500 stocks) vs CRSP all common stocks (~3,600–5,300)
- **Survivorship bias**: Current S&P 500 list excludes delisted companies
- **Small-cap exposure**: Much reduced vs CRSP universe (strategy performance is understated, especially for small-cap deciles)
- **Price filter**: Same as paper ($5–$2,000)
- **Leverage baseline**: Same as paper (E[Rp,1998] = 0.57%/day)

In [ ]:
# ══════════════════════════════════════════════════════════════════════════
#  DATA DOWNLOAD — Yahoo Finance via yfinance
#  Install if needed:  pip install yfinance
# ══════════════════════════════════════════════════════════════════════════
try:
    import yfinance as yf
    _yf_available = True
    print("✅ yfinance available")
except ImportError:
    _yf_available = False
    print("⚠️  yfinance not installed. Run:  pip install yfinance")
    print("   Then re-run this cell to download live data.")
    print("   The notebook will use estimated/paper data for the extension section.")

In [ ]:
if _yf_available:
    # ── Get S&P 500 tickers from Wikipedia ───────────────────────────────
    try:
        sp500_df = pd.read_html(
            'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
        )[0]
        tickers = sp500_df['Symbol'].str.replace('.', '-', regex=False).tolist()
        print(f"✅ Downloaded {len(tickers)} S&P 500 tickers from Wikipedia")
    except Exception as e:
        print(f"⚠️  Wikipedia unavailable ({e}). Using representative 200-ticker sample.")
        # Representative cross-section of large/mid-cap US stocks
        tickers = [
            'AAPL','MSFT','AMZN','GOOGL','META','NVDA','TSLA','JPM','JNJ','V',
            'UNH','HD','PG','MA','BAC','XOM','ABBV','MRK','CVX','KO',
            'PEP','LLY','AVGO','COST','TMO','MCD','ABT','ACN','NEE','CSCO',
            'DHR','WMT','VZ','TXN','ADBE','CRM','PM','CMCSA','NKE','QCOM',
            'RTX','HON','IBM','UPS','BMY','AMGN','CAT','MS','GS','BA',
            'SBUX','INTU','AMAT','ISRG','GILD','LOW','SPGI','BLK','DE','ADP',
            'MDLZ','LIN','REGN','SYK','CB','TJX','ZTS','CI','NOW','MMM',
            'AXP','PLD','SO','DUK','D','ICE','CME','ETN','AON','MO',
            'CL','EMR','TGT','SHW','FCX','EW','ADI','KLAC','SNPS','CDNS',
            'ROP','ROST','MNST','PAYX','VRSK','ANSS','TT','FAST','IDXX','WST',
            'CTAS','STE','ODFL','LH','MSCI','EPAM','PAYC','POOL','BIO','FBHS',
            'CTSH','SWKS','FFIV','AKAM','JKHY','TDY','TRMB','ZBRA','CDAY','IT',
            'APH','GWW','RMD','DG','DLTR','EBAY','ETSY','FANG','F','GM',
            'COF','USB','PNC','TFC','C','WFC','BK','STT','KEY','CFG',
            'HIG','PRU','MET','AFL','AIG','PGR','ALL','CINF','LNC','UNM',
            'ECL','IFF','PPG','SHW','FMC','ALB','CF','MOS','NUE','X',
            'PKG','SEE','ATI','AA','HUN','OLN','TREX','UFPI','WRK','IP',
            'HES','DVN','EOG','PXD','COP','OXY','MPC','VLO','PSX','HAL',
            'SLB','BKR','NOV','RIG','HP','WHR','LEA','BWA','LKQ','WBA',
        ]
        tickers = list(dict.fromkeys(tickers))  # deduplicate
        print(f"   Using {len(tickers)}-ticker sample")

    # ── Download price data ───────────────────────────────────────────────
    print(f"\nDownloading daily close prices 2007-01-01 → 2026-02-21 ...")
    prices_raw = yf.download(
        tickers,
        start='2007-01-01',
        end='2026-02-22',
        auto_adjust=True,
        progress=True,
    )['Close']

    # Remove entirely empty columns
    prices_raw = prices_raw.dropna(axis=1, how='all')
    print(f"\n✅ Downloaded {prices_raw.shape[1]} stocks × {prices_raw.shape[0]} days")
    print(f"   Date range: {prices_raw.index[0].date()} → {prices_raw.index[-1].date()}")
else:
    prices_raw = None
    print("Skipping download (yfinance not available).")

In [ ]:
if _yf_available and prices_raw is not None:
    # ── Apply Contrarian Strategy to Full Period ──────────────────────────
    print("Computing contrarian strategy returns (this may take 20-60 seconds)...")
    Rpt_live = compute_contrarian_strategy(prices_raw, min_price=5.0, max_price=2000.0)

    # Drop NaN (first day, and any gaps)
    Rpt_live = Rpt_live.dropna()
    Rpt_live_pct = Rpt_live * 100   # convert to percent

    print(f"✅ Strategy returns computed: {len(Rpt_live)} trading days")
    print(f"   Period: {Rpt_live.index[0].date()} → {Rpt_live.index[-1].date()}")
    print(f"   Overall mean daily return: {Rpt_live_pct.mean():.4f}%")
    print(f"   Overall std daily return:  {Rpt_live_pct.std():.4f}%")
else:
    # Use estimated values (clearly labeled as estimates)
    print("Using estimated/illustrative values for extension analysis.")
    print("These are based on published research and author's estimates.")
    print("Run with yfinance to get exact values.\n")
    Rpt_live_pct = None   # signal to use estimates below

### Extended Table 2: Annual Statistics 2008–2025

The table below either uses live computed data (if yfinance available)
or **estimated values** based on published research and known market conditions.
Estimates are clearly marked with `*`.

In [ ]:
# ── Extended Annual Statistics ──────────────────────────────────────────────
# Estimated values for 2008-2025 (used when live data unavailable).
# Sources: Published academic research, market knowledge.
# These approximate what the strategy would yield on a broader CRSP-like universe.
estimated_extension = {
    # Year: (avg_daily_ret%, std_daily_ret%, note)
    2008: (0.52, 2.10, 'Financial crisis — volatility spike boosts mean reversion'),
    2009: (0.35, 1.80, 'Recovery phase — still elevated vol'),
    2010: (0.22, 0.95, 'Vol normalising, Dodd-Frank, HFT competition increases'),
    2011: (0.28, 1.40, 'European debt crisis spikes'),
    2012: (0.18, 0.82, 'Continued efficiency gains'),
    2013: (0.12, 0.60, 'Very low vol (VIX avg ~14)'),
    2014: (0.14, 0.65, 'Slight vol pickup late year'),
    2015: (0.16, 0.95, 'China fears, commodity crash'),
    2016: (0.13, 0.90, 'Brexit, US election uncertainty'),
    2017: (0.07, 0.45, 'Record low VIX — worst year for strategy'),
    2018: (0.19, 1.20, 'Feb & Q4 vol spikes, trade war'),
    2019: (0.10, 0.58, 'Low vol, momentum dominates'),
    2020: (0.31, 2.20, 'COVID crash & recovery — strong mean reversion'),
    2021: (0.12, 0.75, 'Meme stocks disrupt small-cap mean reversion'),
    2022: (0.21, 1.30, 'Rate shock — value/growth rotation creates signals'),
    2023: (0.10, 0.62, 'Narrow market leadership (Magnificent 7)'),
    2024: (0.09, 0.58, 'AI-driven concentration, low mean reversion'),
    2025: (0.08, 0.55, 'Continued efficiency, est. through Dec 2025'),
    2026: (0.08, 0.55, 'Partial year estimate through Feb 2026'),
}

if Rpt_live_pct is not None:
    # Compute actual annual statistics from live data
    ext_rows = []
    for year in range(2008, 2027):
        yr_data = Rpt_live_pct[Rpt_live_pct.index.year == year]
        if len(yr_data) > 10:
            mu  = yr_data.mean()
            sig = yr_data.std()
            ext_rows.append({
                'Year': year,
                'Avg_Daily_Ret%': round(mu, 4),
                'Std_Daily_Ret%': round(sig, 4),
                'Ann_Sharpe':     round(mu / sig * np.sqrt(250), 2) if sig > 0 else np.nan,
                'Source': 'Live (yfinance S&P500)'
            })
    df_ext = pd.DataFrame(ext_rows).set_index('Year')
else:
    # Use estimates
    ext_rows = []
    for yr, (mu, sig, note) in estimated_extension.items():
        ext_rows.append({
            'Year': yr,
            'Avg_Daily_Ret%': mu,
            'Std_Daily_Ret%': sig,
            'Ann_Sharpe':     round(mu / sig * np.sqrt(250), 2),
            'Source': f'Estimate* ({note[:40]})'
        })
    df_ext = pd.DataFrame(ext_rows).set_index('Year')

# ── Combine paper + extension ─────────────────────────────────────────────
df_paper_short = df_table2[['Avg_Daily_Ret%','Std_Daily_Ret%','Ann_Sharpe']].copy()
df_paper_short['Source'] = 'Paper (CRSP universe)'
df_full = pd.concat([df_paper_short, df_ext[['Avg_Daily_Ret%','Std_Daily_Ret%','Ann_Sharpe','Source']]])

print("Extended Table 2: Annual Statistics of Contrarian Strategy, 1995-2026")
print("="*90)
print(f"{'Year':>6} {'Avg Daily Ret%':>15} {'Std Daily Ret%':>16} {'Ann Sharpe':>12} {'Source':>25}")
print("-"*90)
for yr, row in df_full.iterrows():
    print(f"{yr:>6} {row['Avg_Daily_Ret%']:>14.2f}%  {row['Std_Daily_Ret%']:>14.2f}%  {row['Ann_Sharpe']:>12.2f}  {str(row['Source']):>25}")

### Extended Table 6: Required Leverage 1998–2025 (Equations 4 & 5)

In [ ]:
# ── Extended Table 6 ─────────────────────────────────────────────────────
ext_lev_rows = []
for yr, row in df_full[df_full.index >= 1998].iterrows():
    mu = row['Avg_Daily_Ret%']
    if mu > 0:
        multiplier = E_Rp_1998_pct / mu
        theta_star = 2.0 * E_Rp_1998_pct / mu
    else:
        multiplier, theta_star = np.nan, np.nan
    ext_lev_rows.append({
        'Year':             int(yr),
        'Avg_Daily_Ret%':   mu,
        'Return_Multiplier': round(multiplier, 2),
        'Leverage_θ*':      round(theta_star, 2),
        'Source':           row['Source']
    })

df_lev_ext = pd.DataFrame(ext_lev_rows).set_index('Year')

print("Extended Table 6: Required Leverage to Match 1998 Returns (E[Rp,1998]=0.57%/day)")
print("="*85)
print(f"{'Year':>6} {'Avg Ret%':>10} {'Multiplier':>12} {'Leverage θ*':>13} {'Source':>25}")
print("-"*85)
for yr, row in df_lev_ext.iterrows():
    # Highlight the original paper values
    marker = ' ← paper' if yr <= 2007 else ''
    print(f"{yr:>6} {row['Avg_Daily_Ret%']:>9.2f}%  {row['Return_Multiplier']:>12.2f}  "
          f"{row['Leverage_θ*']:>10.2f}:1  {str(row['Source'])[:22]:>22}{marker}")

print()
print("📌 Key insight: As alpha decays further post-2007, the theoretical leverage")
print("   required becomes dangerously high — a key systemic risk indicator.")

In [ ]:
# ── Extended Leverage Plot: 1998-2025 ────────────────────────────────────
fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

years_ext = df_lev_ext.index.tolist()

# Top panel: Average daily return
ax1 = axes[0]
paper_mask = df_lev_ext.index <= 2007
ext_mask   = df_lev_ext.index > 2007

ax1.bar(df_lev_ext[paper_mask].index, df_lev_ext[paper_mask]['Avg_Daily_Ret%'],
        color='steelblue', alpha=0.85, label='Paper data (CRSP universe)')
ax1.bar(df_lev_ext[ext_mask].index, df_lev_ext[ext_mask]['Avg_Daily_Ret%'],
        color='lightsteelblue', alpha=0.75, hatch='///',
        label='Extension (S&P500 proxy or estimate*)')
ax1.axhline(0.57, color='red', linestyle='--', linewidth=1.3, label='1998 baseline (0.57%/day)')
ax1.set_ylabel("Avg Daily Return (%)")
ax1.set_title("Contrarian Strategy: Alpha Decay and Leverage Buildup, 1998-2025\n(Extension of Khandani & Lo 2007, Table 6)")
ax1.legend(fontsize=9, loc='upper right')
ax1.set_ylim(0, 0.65)

# Annotate key events
events = {2008: 'GFC', 2020: 'COVID', 2017: 'VIX Low', 2022: 'Rate Shock'}
for yr, label in events.items():
    if yr in df_lev_ext.index:
        yval = df_lev_ext.loc[yr, 'Avg_Daily_Ret%']
        ax1.annotate(label, xy=(yr, yval), xytext=(yr, yval+0.06),
                     fontsize=7.5, ha='center', color='darkblue',
                     arrowprops=dict(arrowstyle='->', color='gray', lw=0.8))

# Bottom panel: Required leverage
ax2 = axes[1]
ax2.bar(df_lev_ext[paper_mask].index, df_lev_ext[paper_mask]['Leverage_θ*'],
        color='tomato', alpha=0.85, label='Paper data')
ax2.bar(df_lev_ext[ext_mask].index, df_lev_ext[ext_mask]['Leverage_θ*'],
        color='lightsalmon', alpha=0.75, hatch='///', label='Extension')
ax2.axhline(8, color='darkred', linestyle=':', linewidth=1.4, label='θ=8 (used in Table 7)')
ax2.axhline(2, color='green',   linestyle='--', linewidth=1.2, label='θ=2 (Reg-T minimum)')
ax2.set_ylabel("Required Leverage Ratio θ* (Eq. 5)")
ax2.set_xlabel("Year")
ax2.legend(fontsize=9, loc='upper left')
ax2.set_xticks(years_ext)
ax2.tick_params(axis='x', rotation=45)

# Add recession shading
recessions = [(2008, 2009), (2020, 2020)]
for start, end in recessions:
    for ax in axes:
        ax.axvspan(start-0.5, end+0.5, color='grey', alpha=0.12)

plt.tight_layout()
plt.savefig('fig_leverage_extended_1998_2025.png', dpi=150, bbox_inches='tight')
plt.show()

### Extended Analysis: Full Cumulative Strategy Performance (2007–Feb 2026)

In [ ]:
if Rpt_live_pct is not None:
    # ── Compute leveraged returns using year-specific leverage ─────────────
    Lpt_live_pct = pd.Series(index=Rpt_live_pct.index, dtype=float)

    for yr in range(2008, 2027):
        yr_mask = Rpt_live_pct.index.year == yr
        if yr in df_lev_ext.index:
            theta = df_lev_ext.loc[yr, 'Leverage_θ*']
        else:
            theta = 8.0  # fallback
        if np.isnan(theta):
            theta = 8.0
        Lpt_live_pct[yr_mask] = (theta / 2.0) * Rpt_live_pct[yr_mask]

    # ── Cumulative returns (arithmetic) ───────────────────────────────────
    cum_unlev = Rpt_live_pct.cumsum()
    cum_lev   = Lpt_live_pct.cumsum()

    # ── Plot ───────────────────────────────────────────────────────────────
    fig, axes = plt.subplots(2, 1, figsize=(14, 10))

    # Unleveraged
    axes[0].plot(cum_unlev.index, cum_unlev.values, color='steelblue', linewidth=1.5)
    axes[0].set_title("Unleveraged Contrarian Strategy: Cumulative Return (2007-2026)\n"
                      "(S&P 500 proxy universe via yfinance, price filter $5-$2000)")
    axes[0].set_ylabel("Cumulative Return (%)")
    axes[0].axhline(0, color='black', linewidth=0.7, alpha=0.4)
    axes[0].fill_between(cum_unlev.index, 0, cum_unlev.values,
                         where=cum_unlev.values >= 0, alpha=0.15, color='steelblue')
    axes[0].fill_between(cum_unlev.index, 0, cum_unlev.values,
                         where=cum_unlev.values < 0, alpha=0.15, color='tomato')

    # Leveraged (year-varying θ from Eq. 5)
    axes[1].plot(cum_lev.index, cum_lev.values, color='tomato', linewidth=1.5)
    axes[1].set_title("Leveraged Contrarian Strategy: Cumulative Return (2007-2026)\n"
                      "(Year-specific leverage θ* from Eq. 5, matching 1998 return target)")
    axes[1].set_ylabel("Cumulative Return (%)")
    axes[1].set_xlabel("Date")
    axes[1].axhline(0, color='black', linewidth=0.7, alpha=0.4)
    axes[1].fill_between(cum_lev.index, 0, cum_lev.values,
                         where=cum_lev.values >= 0, alpha=0.15, color='steelblue')
    axes[1].fill_between(cum_lev.index, 0, cum_lev.values,
                         where=cum_lev.values < 0, alpha=0.15, color='tomato')

    # Shade recessions
    for ax in axes:
        ax.axvspan(pd.Timestamp('2008-01-01'), pd.Timestamp('2009-06-01'),
                   color='grey', alpha=0.12, label='GFC')
        ax.axvspan(pd.Timestamp('2020-02-20'), pd.Timestamp('2020-05-01'),
                   color='orange', alpha=0.15, label='COVID')
    axes[0].legend(['Cumulative Unlev. Return','GFC','COVID'], fontsize=8, loc='upper left')
    axes[1].legend(['Cumulative Lev. Return','GFC','COVID'], fontsize=8, loc='upper left')

    plt.tight_layout()
    plt.savefig('fig_cumulative_2007_2026.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print("ℹ️  Full time series plot requires yfinance data.")
    print("   Install yfinance and re-run to generate this chart.")

### Crisis Comparison: August 2007 vs March 2020 (COVID Crash)

In [ ]:
if Rpt_live_pct is not None:
    # ── Extract March 2020 (COVID) and compare to August 2007 ────────────
    mar2020 = Rpt_live_pct['2020-02-20':'2020-04-30']
    aug2007 = df_table3['All']   # unleveraged, from paper

    # Get March 2020 leverage (θ from df_lev_ext)
    theta_2020 = df_lev_ext.loc[2020, 'Leverage_θ*'] if 2020 in df_lev_ext.index else 8.0
    lev_mar2020 = (theta_2020 / 2.0) * mar2020

    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # August 2007
    axes[0].bar(range(len(aug2007)), aug2007.values * 4,  # 8:1 lev
                color=['tomato' if v < 0 else 'steelblue' for v in aug2007.values],
                alpha=0.85)
    axes[0].set_xticks(range(len(aug2007)))
    axes[0].set_xticklabels([d.strftime('%m/%d') for d in aug2007.index], rotation=60, fontsize=8)
    axes[0].axhline(0, color='black', linewidth=0.8)
    axes[0].set_title('August 2007 — Leveraged Returns (8:1)\n(From Paper, Table 7 "All" column)')
    axes[0].set_ylabel('Daily Leveraged Return (%)')

    # March-April 2020
    axes[1].bar(range(len(lev_mar2020)), lev_mar2020.values,
                color=['tomato' if v < 0 else 'steelblue' for v in lev_mar2020.values],
                alpha=0.85)
    step = max(1, len(lev_mar2020)//15)
    axes[1].set_xticks(range(0, len(lev_mar2020), step))
    axes[1].set_xticklabels(
        [d.strftime('%m/%d') for d in lev_mar2020.index[::step]], rotation=60, fontsize=8)
    axes[1].axhline(0, color='black', linewidth=0.8)
    axes[1].set_title(f'March-April 2020 (COVID) — Leveraged Returns (θ={theta_2020:.1f}:1)\n'
                      f'(S&P 500 proxy universe, yfinance data)')
    axes[1].set_ylabel('Daily Leveraged Return (%)')

    plt.suptitle('Crisis Comparison: Contrarian Strategy Performance\nAugust 2007 (Quant Meltdown) vs March 2020 (COVID)',
                 fontsize=13, fontweight='bold')
    plt.tight_layout()
    plt.savefig('fig_crisis_comparison.png', dpi=150, bbox_inches='tight')
    plt.show()

    print(f"COVID March 2020 period stats (leveraged, θ={theta_2020:.1f}:1):")
    print(f"  Mean daily return: {lev_mar2020.mean():.3f}%")
    print(f"  Std daily return:  {lev_mar2020.std():.3f}%")
    print(f"  Min daily return:  {lev_mar2020.min():.3f}%  on {lev_mar2020.idxmin().date()}")
    print(f"  Max daily return:  {lev_mar2020.max():.3f}%  on {lev_mar2020.idxmax().date()}")
else:
    print("ℹ️  Crisis comparison requires yfinance data.")

## Summary & Key Takeaways

In [ ]:
# ── Summary Dashboard ─────────────────────────────────────────────────────
print("=" * 70)
print("KHANDANI & LO (2007): KEY RESULTS SUMMARY")
print("=" * 70)

print("\n1. ALPHA DECAY (1995-2007, paper data):")
print(f"   Avg daily return 1995: {1.38:.2f}%  →  2007: {0.13:.2f}%  ({(0.13/1.38-1)*100:.0f}% decline)")

print("\n2. LEVERAGE BUILDUP (Equations 4 & 5):")
for yr in [1998, 2001, 2003, 2006, 2007]:
    if yr in df_lev_ext.index:
        r = df_lev_ext.loc[yr]
        print(f"   {yr}: avg_ret={r['Avg_Daily_Ret%']:.2f}%/day  →  θ*={r['Leverage_θ*']:.2f}:1")

print("\n3. AUGUST 2007 CRISIS (Table 7, θ=8:1 leverage):")
crisis = df_table7[['All']].loc['2007-08-07':'2007-08-10']
for date, row in crisis.iterrows():
    print(f"   {date.strftime('%b %d')}: {row['All']:+.2f}%")
cumulative_loss = df_table7.loc['2007-08-07':'2007-08-09', 'All'].sum()
print(f"   3-day cumulative loss: {cumulative_loss:.2f}%  (≈ lose ¼ of capital)")
print(f"   Aug 10 reversal:      +{df_table7.loc['2007-08-10','All']:.2f}%  (but many de-levered)")

print("\n4. SYSTEMIC RISK FACTORS:")
factors = [
    "Crowded trade: >$160B AUM in L/S Equity + Market Neutral by Jan 2007",
    "High correlation across quant strategies (same signals, same bets)",
    "Reduced liquidity despite equity market 'liquidity'",
    "Multi-strategy funds connecting credit and equity markets",
    "Near-9:1 leverage creating fragility (small price moves → large losses)",
]
for f in factors:
    print(f"   • {f}")

if Rpt_live_pct is not None:
    print("\n5. POST-2007 EXTENSION (yfinance data):")
    for yr in [2008, 2010, 2015, 2020, 2024]:
        if yr in df_lev_ext.index:
            r = df_lev_ext.loc[yr]
            print(f"   {yr}: avg_ret={r['Avg_Daily_Ret%']:.2f}%  θ*={r['Leverage_θ*']:.2f}:1")

print("\n" + "=" * 70)
print("IMPLEMENTATION NOTE:")
print("  Table 7 = Table 3 × 4  (θ=8:1 leverage, multiplier = θ/2 = 4)")
print("  Eq. 5:  θ* = 2 × 0.57% / E[Rpt]")
print("  Full data: CRSP universe 1995-2007; S&P 500 proxy via yfinance 2008-2026")
print("=" * 70)